# Neural Network for MNIST database recognizing

This is a Neural Network developed without Machine Learning frameworks, using `numpy` and `matplotlib`, only for learning purposes. The objective of this project is practice Neural Networks Design principles and enhance knowledges on Machine Learning and Deep Learning.


## 1 - Library imports

In [ ]:
import mnist
import numpy as np
import matplotlib.pyplot as plt
import os
from enum import Enum, unique
np.seterr(all='raise')

## 2 - MNIST Datasets download

In [ ]:
if(not os.path.isdir("./datasets/")):
    os.mkdir("./datasets")
mnist.download_file("train-images-idx3-ubyte.gz", "./datasets")
mnist.download_file("train-labels-idx1-ubyte.gz", "./datasets")
mnist.download_file("t10k-images-idx3-ubyte.gz", "./datasets")
mnist.download_file("t10k-labels-idx1-ubyte.gz", "./datasets")

## 3 - MNIST Datasets Load

In [ ]:
train_images_raw = mnist.train_images()
train_labels_raw = mnist.train_labels()
test_images_raw = mnist.test_images()
test_labels_raw = mnist.test_labels()

index = int(np.round(np.random.rand()*len(train_images_raw)))

train_images = np.transpose(train_images_raw.reshape(train_images_raw.shape[0], train_images_raw.shape[1]*train_images_raw.shape[2]))
train_labels = np.array([np.binary_repr(i, width=4)for i in train_labels_raw])
train_labels = np.transpose(np.array([[int(i[0]), int(i[1]), int(i[2]), int(i[3])] for i in train_labels]))
test_images = np.transpose(test_images_raw.reshape(test_images_raw.shape[0], test_images_raw.shape[1]*test_images_raw.shape[2]))
test_labels = np.array([np.binary_repr(i, width=4)for i in test_labels_raw])
test_labels = np.transpose(np.array([[int(i[0]), int(i[1]), int(i[2]), int(i[3])] for i in test_labels]))

print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

plt.imshow(train_images_raw[index], cmap='magma')
print(train_labels_raw[index])
plt.show()

## 4 - Activation functions enum

In [ ]:
@unique
class Activation(Enum):
    SIGMOID = 1
    TANH = 2
    RELU = 3
    LEAKY_RELU = 4

## 5 - Hyperparameters

In [ ]:
FEATURES = train_images.shape[0]
LAYERS = 3
LAYER_UNITS = np.array([16, 10, 4], dtype=np.uint32)
LAYER_ACTIVATIONS = np.array([Activation['RELU'], Activation['RELU'], Activation['RELU']])
ALPHA = 10e-2
LAMBDA_REG = 10e-3
ITERATIONS = 200
EPSILON = 0.05

EXAMPLES = train_images.shape[1]

RANDOM_FACTOR_X = 82
RANDOM_FACTOR_W = 0.01
RANDOM_FACTOR_B = 3

## 6 - Activation functions and it's derivatives

In [ ]:
LEAKY_RELU_MULTIPLIER = 0.01

def linear_func(X_matrix, W_matrix, b_array):
    return np.dot(W_matrix, X_matrix) + b_array

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def sigmoid_threshold(z):
    return np.round(z)

def tanh(z):
    return np.tanh(z)

def relu(z):
    return np.maximum(0, z)

def leaky_relu(z):
    return np.maximum(LEAKY_RELU_MULTIPLIER * z, z)

def derivative_sigmoid(z):
    return sigmoid(z) * (1 - sigmoid(z))

def derivative_tanh(z):
    return 1 - (tanh(z)**2)

def derivative_relu(z):
    if(z < 0):
        return 0
    else:
        return 1

def derivative_leaky_relu(z):
    if(z < 0):
        return LEAKY_RELU_MULTIPLIER
    else:
        return 1

derivative_relu = np.vectorize(derivative_relu)
derivative_leaky_relu = np.vectorize(derivative_leaky_relu)

## 7 - Normalization functions

In [ ]:
def normalize_dataset(X_matrix):
    X_norm = np.zeros(X_matrix.shape)
    for i in range(X_matrix.shape[0]):
        mu = np.mean(X_matrix[i])
        sigma = np.std(X_matrix[i])
        X_norm[i] = (X_matrix[i] - mu)/sigma
    return X_norm

def normalize_input(x_input, X_matrix):
    x_norm = np.zeros(x_input.shape)
    for i in range(X_matrix.shape[0]):
        mu = np.mean(X_matrix[i])
        sigma = np.std(X_matrix[i])
        x_norm = (x_input - mu)/sigma
    return x_norm

## 8 - Forward and Backward Propagation Functions

In [ ]:
def fwd_prop(A_previous, W_layer, b_layer, activationType = Activation['SIGMOID']):
    Z_layer = linear_func(A_previous, W_layer, b_layer)
    if (activationType == Activation['SIGMOID']):
        A_layer = sigmoid(Z_layer)
    elif (activationType == Activation['TANH']):
        A_layer = tanh(Z_layer)
    elif (activationType == Activation['RELU']):
        A_layer = relu(Z_layer)
    elif (activationType == Activation['LEAKY_RELU']):
        A_layer = leaky_relu(Z_layer)
    else:
        A_layer = sigmoid(Z_layer)
    return A_layer, Z_layer

def back_prop(dA_layer, A_previous, Z_layer, W_layer, b_layer, activationType = Activation['SIGMOID']):
    if (activationType == Activation['SIGMOID']):
        dZ_layer = np.multiply(dA_layer, derivative_sigmoid(Z_layer))
    elif (activationType == Activation['TANH']):
        dZ_layer = np.multiply(dA_layer, derivative_tanh(Z_layer))
    elif (activationType == Activation['RELU']):
        dZ_layer = np.multiply(dA_layer, derivative_relu(Z_layer))
    elif (activationType == Activation['LEAKY_RELU']):
        dZ_layer = np.multiply(dA_layer, derivative_leaky_relu(Z_layer))
    else:
        dZ_layer = np.multiply(dA_layer, derivative_sigmoid(Z_layer))
    dW_layer = np.dot(dZ_layer, np.transpose(A_previous))/dZ_layer.shape[1]
    db_layer = np.sum(dZ_layer, axis = 1, keepdims = True)/dZ_layer.shape[1]
    dA_previous = np.dot(np.transpose(W_layer), dZ_layer)
    return dA_previous, dW_layer, db_layer   

## 9 - Predict Function

In [ ]:
def predict(X, W, b,
            layers = LAYERS,
            layer_activations = LAYER_ACTIVATIONS):
    A = X
    for i in range(1, layers+1):
        A, z = fwd_prop(A, W[i], b[i], layer_activations[i-1])
    return A

## 10 - Cost function

In [ ]:
def cost(X, y, W, b, lambda_reg = LAMBDA_REG):
    y_prediction = predict(X, W, b)
    regularization = 0
    try:
        loss = - ((np.dot(y, np.transpose(np.log(y_prediction)))) + np.dot((1 - y), np.transpose(np.log(1 - y_prediction))))
    except FloatingPointError:
        y_prediction = abs(y_prediction-0.00000001)
        loss = - ((np.dot(y, np.transpose(np.log(y_prediction)))) + np.dot((1 - y), np.transpose(np.log(1 - y_prediction))))
    for i in W:
        regularization += np.sum(i**2)
    cost = (np.sum(loss)/y.shape[1]) + ((lambda_reg/(2*y.shape[1]))*regularization)
    return cost

## 11 - Fit Function

In [ ]:
def fit(X, y,
        features = FEATURES,
        layers = LAYERS,
        layer_units = LAYER_UNITS,
        layer_activations = LAYER_ACTIVATIONS,
        examples = EXAMPLES,
        alpha = ALPHA,
        lambda_reg = LAMBDA_REG,
        iterations = ITERATIONS,
        epsilon = EPSILON):
    
    W = {1: np.random.randn(layer_units[0], features)}
    dW = {1: np.zeros([layer_units[0], features])}
    b = {1:np.random.randn(layer_units[0], 1)}
    db = {1: np.zeros([layer_units[0], 1])}
    Z = {0: X}
    A = {0: X}
    dA = {0: np.array([])}
    for k in range(layers - 1):
        W[k+2] = np.random.randn(layer_units[k+1], layer_units[k])
        dW[k+2] = np.zeros([layer_units[k+1], layer_units[k]])
        b[k+2] = np.random.randn(layer_units[k+1], 1)
        db[k+2] = np.zeros([layer_units[k+1], 1])
        Z[k+1] = np.zeros([layer_units[k+1], examples])
        A[k+1] = np.zeros([layer_units[k+1], examples])
        dA[k+1] = np.zeros([layer_units[k+1], examples])

    for i in range(iterations):

        for j in range(layers):
            A[j+1], Z[j+1] = fwd_prop(A[j], W[j+1], b[j+1], layer_activations[j])
            
        dA[layers] = A[layers] - y

        for j in range(layers-1, -1, -1):
            dA[j], dW[j+1], db[j+1] = back_prop(dA[j+1], A[j], Z[j+1], W[j+1], b[j+1], layer_activations[j])

        for j in range(1, layers+1):
            W[j] = (W[j] * (1 - ((alpha*lambda_reg)/y.shape[1]))) - (alpha*dW[j])
            b[j] = b[j] - (alpha*db[j])      

    return W, b

## 12 - Training the N.N.

In [ ]:
W_final, b_final = fit(train_images, train_labels)

## 13 - Testing the N.N.